In [2]:
import sqlite3
import itertools

In [67]:
import numpy
print(numpy.__version__)

1.9.2


In [40]:
mood_tag_list = set(['MD', 'JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBPZ'])

In [113]:
conn = sqlite3.connect('C:/KJ/Temp/reviews.db')
c = conn.cursor()
p_list = []
n_list = []
neutral_list = []
for row in itertools.islice(c.execute('SELECT SNIPPET FROM REVIEWS WHERE RATING>4.2 AND SNIPPET IS NOT NULL'),1000):
    p_list.append((row[0],'positive'))
print (len(p_list))

for row in itertools.islice(c.execute('SELECT SNIPPET FROM REVIEWS WHERE RATING<3.2 AND SNIPPET IS NOT NULL'),1000):
    n_list.append((row[0],'negative')) 
print (len(n_list))

#for row in itertools.islice(c.execute('SELECT SNIPPET FROM REVIEWS WHERE RATING>=3.2 and RATING<=4.2 AND SNIPPET IS NOT NULL'),1000):
#    neutral_list.append((row[0],'neutral'))
#print (len(neutral_list))


1000
1000


In [114]:
reviews = []
for (words, sentiment) in p_list + n_list + neutral_list :
    words_filtered = [e.lower() for e in get_mood_words(words) if len(e) > 1] 
    reviews.append((words_filtered, sentiment))    
print (len(reviews))

2000


In [104]:
def get_words_in_reviews(reviews):
    all_words = []
    for (words, sentiment) in reviews:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [105]:
word_features = get_word_features(get_words_in_reviews(reviews))    
training_set = nltk.classify.apply_features(extract_features, reviews)   
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [111]:
test_list = []
for row in itertools.islice(c.execute('SELECT SNIPPET FROM POS_REVIEWS WHERE RATING >4.2 AND SNIPPET IS NOT NULL'),50):
    test_list.append((row[0],'positive'))
for row in itertools.islice(c.execute('SELECT SNIPPET FROM NEG_REVIEWS WHERE RATING <3.2 AND SNIPPET IS NOT NULL'),50):
    test_list.append((row[0],'negative'))
#for row in itertools.islice(c.execute('SELECT SNIPPET FROM NEG_REVIEWS WHERE RATING >=3.2 AND RATING<=4.2 AND SNIPPET IS NOT NULL'),50):
#    test_list.append((row[0],'neutral'))
#print (len(test_list))

success = 0
failed_list = []
for (rev,tagged_mood) in test_list:
    predicted_mood = classifier.classify(extract_features(rev.split()))
    if predicted_mood==tagged_mood:
        success+=1
    else:
        failed_list.append((predicted_mood, tagged_mood, rev))
print (success/len(test_list))
print (failed_list)

0.7
[('neutral', 'positive', 'Chill spot w good music, good vibes and good drink selection. \n\nLots of drafts, a few cocktails, cider. Snacks, trivia night, hot dog night, etc.. HH 1-7...'), ('negative', 'positive', 'I LOVE OLIVER! Our server/bartender was THE BEST! \n\nWe ordered the wings, bacon bowl, and bacon/cheese tator tots... and then we ordered the wings and tator...'), ('negative', 'positive', 'Nice local bar with a really good tap selection. Cool rare ciders for the girlfriend too. The bartenders here are friendly and really know their beer,...'), ('negative', 'positive', "I went to watch Nutcracker Rouge with a friend recently.  Ugh, it's like the best thing ever.  Go watch it now!\n\nSo we get to this place, it's half hidden..."), ('neutral', 'positive', 'This is a "need to discover" hidden little gem of a bar in Bushwick east of Williamsburg. Johnny is not only a super bar tender with cocktails that...'), ('negative', 'positive', 'Stopped in on a Friday about a month ago

In [112]:
print (classifier.show_most_informative_features(32))

Most Informative Features
          contains(cute) = True           neutra : negati =      8.3 : 1.0
        contains(unique) = True           positi : negati =      7.0 : 1.0
          contains(knew) = True           positi : negati =      6.3 : 1.0
       contains(totally) = True           positi : negati =      6.3 : 1.0
    contains(pleasantly) = True           negati : positi =      6.3 : 1.0
         contains(clean) = True           negati : neutra =      6.2 : 1.0
       contains(exactly) = True           neutra : negati =      5.7 : 1.0
         contains(drive) = True           positi : neutra =      5.7 : 1.0
          contains(loud) = True           neutra : negati =      5.7 : 1.0
            contains(as) = True           negati : neutra =      5.7 : 1.0
          contains(dead) = True           negati : neutra =      5.7 : 1.0
       contains(average) = True           negati : neutra =      5.7 : 1.0
          contains(true) = True           positi : negati =      5.7 : 1.0

In [30]:
from nltk.tokenize import word_tokenize
text = word_tokenize("And now for something completely different")
nltk.pos_tag(text)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

In [45]:
def get_mood_words(text):
    mood_tag_list = set(['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBP', 'VBPZ'])
    mood_words = []
    tagged_words = nltk.pos_tag(word_tokenize(text))    
    for word, pos in tagged_words:        
        if pos in mood_tag_list:
            mood_words.append(word) 
    return mood_words
print (get_mood_words("And now for something completely different"))

['now', 'completely', 'different']
